In [6]:
#pip install tqdm
#!pip install -U scikit-learn
#!pip install seaborn

In [1]:
import requests
import urllib.request
from tqdm import tqdm
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sklearn

gameNum = 10000

#url = 'https://opgg-data-analytics.s3.ap-northeast-2.amazonaws.com/recruit/20200910/lol_kr_v1018_soloRank.json'


In [3]:
#response = urllib.request.urlopen(url)

In [4]:
#urllib.request.urlretrieve(url,'opggData')

In [2]:
data = []
iter = 0
for line in tqdm(open('opggData', 'r')):
    data.append(json.loads(line))
    iter+=1
    if iter==gameNum:
        break

9999it [00:13, 756.24it/s]


In [3]:
data[0].keys()

dict_keys(['teams', 'gameid', 'queueid', 'gameversion', 'gametype', 'platformid', 'participantidentities', 'seasonid', 'mapid', 'gameduration', 'gamemode', 'gamecreation', 'participants', 'participantextendedstats'])

In [7]:
data[0]['participantidentities'][0]['player']['summonerid']

'PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA'

In [4]:
data[0]['gamecreation']

1599158094

In [8]:
#선택한 포지션
# for i in range(10) :
#     if i%5==0 :
#         print()
#     print(data[1]['participantextendedstats'][i]['position'])

In [9]:
#타임라인에 따른 포지션
#for i in range(0,10) :
#    if i==5 :
#        print()
#    print(data[1]['participants'][i]['timeline']['lane'])

In [10]:
'''
for i in range(10):
    if i%5==0 :
        print()
    print(championDict[data[1]['participants'][i]['championid']])
'''

"\nfor i in range(10):\n    if i%5==0 :\n        print()\n    print(championDict[data[1]['participants'][i]['championid']])\n"

In [11]:
#data[0]['teams'][0]['bans']

In [12]:
#data[0]['participants'][0].keys()

In [13]:
itemURL = 'http://ddragon.leagueoflegends.com/cdn/10.18.1/data/en_US/item.json'
request = requests.get(itemURL)
itemData = request.json()

In [14]:
itemDict = {}
for i_name in itemData['data'].keys() :
    itemDict[int(i_name)]=itemData['data'][i_name]['name']

In [15]:
itemDict[3123]

"Executioner's Calling"

In [16]:
championURL = 'http://ddragon.leagueoflegends.com/cdn/10.23.1/data/en_US/champion.json'
request = requests.get(championURL)

In [17]:
championData = request.json()
request.json().keys()

dict_keys(['format', 'type', 'version', 'data'])

In [18]:
championDict = {}
for c_name in championData['data'].keys() :
    championDict[int(championData['data'][c_name]['key'])]=c_name

In [19]:
championDict[58]

'Renekton'

In [20]:
data[0]['participants'][0]['stats'].keys()

dict_keys(['unrealkills', 'deaths', 'item2', 'wardskilled', 'champlevel', 'perk1var3', 'magicdamagedealttochampions', 'inhibitorkills', 'totalscorerank', 'statperk0', 'neutralminionskilled', 'combatplayerscore', 'totaldamagetaken', 'firsttowerkill', 'objectiveplayerscore', 'perk0var2', 'perk3var2', 'perk2var2', 'item0', 'firsttowerassist', 'perk5var2', 'totaltimecrowdcontroldealt', 'timeccingothers', 'totalunitshealed', 'perk3var1', 'totaldamagedealttochampions', 'goldspent', 'visionwardsboughtingame', 'killingsprees', 'perk2var3', 'visionscore', 'perk5', 'item1', 'perkprimarystyle', 'perk4var3', 'largestmultikill', 'perk4var1', 'longesttimespentliving', 'perk5var1', 'perk4', 'totalheal', 'perk3var3', 'perk2var1', 'statperk1', 'perksubstyle', 'perk5var3', 'truedamagetaken', 'magicaldamagetaken', 'physicaldamagedealt', 'largestcriticalstrike', 'totalplayerscore', 'wardsplaced', 'largestkillingspree', 'totaldamagedealt', 'item4', 'physicaldamagetaken', 'kills', 'triplekills', 'damagedeal

In [21]:
import math
assist = []
kill = []
death = []
kda = []
gold = []
level = []
champion_num = []
summoner_id = []

items = [ [] for _ in range(7)]

for gameCnt in tqdm(range(gameNum)) : 
    time = data[gameCnt]['gameduration'] / 60
    for i in range(10) :
        champion_num.append(data[gameCnt]['participants'][i]['championid'])
        assist.append(data[gameCnt]['participants'][i]['stats']['assists'] /time )
        kill.append(data[gameCnt]['participants'][i]['stats']['kills'] /time )
        death.append(data[gameCnt]['participants'][i]['stats']['deaths'] /time )
        gold.append(data[gameCnt]['participants'][i]['stats']['goldearned'] / time)
        level.append(data[gameCnt]['participants'][i]['stats']['champlevel'] / time)
        summoner_id.append(data[gameCnt]['participantidentities'][i]['player']['summonerid'])

        for item_num in range(7) :
            items[item_num].append(data[gameCnt]['participants'][i]['stats']['item'+str(item_num)])
        
        if death[len(death)-1] == 0 :
            kda.append(1.5 * (assist[len(assist)-1] + kill[len(kill)-1]))
        else :
            kda.append((assist[len(assist)-1] + kill[len(kill)-1]) / death[len(death)-1])


#assistCnt = [0 for _ in range(int(max(assist)*100)+2)]
#killCnt = [0 for _ in range(int(max(kill)*100)+2)]
#deathCnt = [0 for _ in range(int(max(death)*100)+2)]
#kdaCnt = [0 for _ in range(int(max(kda)*10)+2)]
#goldCnt = [0 for _ in range(int(max(gold))+2)]

100%|██████████| 10000/10000 [00:01<00:00, 5030.10it/s]


In [22]:
# for item in range(len(assist)) :
#     assistCnt[int(assist[item] * 100)] +=1
#     killCnt[int(kill[item] * 100)] +=1
#     deathCnt[int(death[item] * 100)] +=1
#     kdaCnt[int(kda[item] * 10)] +=1
#     goldCnt[int(gold[item])] +=1


In [23]:
# from matplotlib import pyplot as plt
# import numpy as np

# x_axis = np.arange(len(killCnt))
# plt.bar(x_axis, killCnt)

In [24]:
# x_axis = np.arange(len(goldCnt))
# plt.bar(x_axis, goldCnt)

In [25]:
kill_df = pd.DataFrame(kill)
death_df = pd.DataFrame(death)
x_data = pd.DataFrame()
x_data['kill'] = kill
x_data['assist'] = assist
x_data['death'] = death
x_data['gold'] = gold
x_data['level'] = level

x_data['troll'] = 0
x_data['championid'] = champion_num
x_data['summoner_id'] = summoner_id

x_data

,kill,assist,death,gold,level,troll,championid,summoner_id
0,0.158520,0.356671,0.118890,359.167768,0.515192,0,38,PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA
1,0.118890,0.475561,0.277411,278.678996,0.435931,0,497,JRuCiFpuzcsnQ9Gadn5A_0bm66-PrFaA3WKP5juhgetrx3Y
2,0.317041,0.158520,0.237781,423.725231,0.515192,0,164,o5ZjlQkLgDZzDG_dkT0GVL7X34kp51abqkUN6P7K2WPeY1dy
3,0.079260,0.277411,0.356671,323.659181,0.475561,0,19,TUdiT-gEHdtkmr9JkhqvtNy5L_nutzsw2oH23kgO71NSsA
4,0.356671,0.198151,0.118890,427.093791,0.515192,0,145,qmcY1AJTdznK_aTgW2e1qwCY1uwbam9HU09Wsa7nI9Wt5sYQ
...,...,...,...,...,...,...,...,...
99995,0.183374,0.293399,0.183374,496.613692,0.550122,0,236,XU_3wTozp_g-sec3SdeBp7WOg61BU4Bsi9P636axectzPRY
99996,0.330073,0.256724,0.183374,437.090465,0.550122,0,245,oYgrmGD1cGqAN6My_talEYVTEp9YpZkjDsIZ3NMhjWqI9g
99997,0.036675,0.513447,0.220049,242.603912,0.440098,0,432,q4uKpVa1RT8cihUV3Wa1t9Y7o-9CttHF3mOYxtdkksSUKQ
99998,0.403423,0.220049,0.220049,416.625917,0.513447,0,55,Q_fw1SGj4BUHxM3gFnRWx7WcchwUTZbN4S-F7W0jwguxi-4


In [26]:
# feature_list = ['kill','assist','death','gold','level']

# for feature in feature_list :
#     plot_x_data = [x_data[feature][x_data['troll']==0],x_data[feature][x_data['troll']==1]]

#     if feature == 'gold' :
#         plt.yticks([-10 + 1 * i for i in range(16)])
#     else :
#         plt.yticks([-10 + 1 * i for i in range(16)])
    
#     flierprops = dict(markerfacecolor='r',markersize=0.1)
#     plt.title(feature)
#     plt.grid()

#     plt.boxplot(plot_x_data, flierprops=flierprops)
#     plt.xticks([1,2] , ['not_afk','afk'])

#     plt.show()


In [27]:
# from scipy import stats

# feature = 'kill'
# stats.f_oneway(x_data[feature][x_data['troll']==1], x_data[feature][x_data['troll']==0], x_data[feature])

In [28]:
# for feature in feature_list :
#     tTestResultDiffVar = stats.ttest_ind(x_data[feature][x_data['troll']==0], x_data[feature][x_data['troll']==1], equal_var=False)

#     print(feature,tTestResultDiffVar)

In [29]:
#챔피언별 피쳐들로 표준 정규화 진행

features = ['kill','assist','death','gold','level']

champion_list = x_data['championid'].unique()

for cham_id in tqdm(champion_list) : 
    for f in features : 
        f_mean = x_data[f][x_data['championid']==cham_id].mean()
        f_std = x_data[f][x_data['championid']==cham_id].std() + math.pow(10,-8)

        x_data[f][x_data['championid']==cham_id] -= f_mean
        x_data[f][x_data['championid']==cham_id] /= f_std

x_data

100%|██████████| 150/150 [00:15<00:00,  9.78it/s]


,kill,assist,death,gold,level,troll,championid,summoner_id
0,-0.343376,1.745007,-0.491769,-0.431260,-0.512853,0,38,PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA
1,1.180590,0.130797,1.246795,0.398971,-0.230212,0,497,JRuCiFpuzcsnQ9Gadn5A_0bm66-PrFaA3WKP5juhgetrx3Y
2,0.929154,-0.218189,0.525580,0.450369,-0.496110,0,164,o5ZjlQkLgDZzDG_dkT0GVL7X34kp51abqkUN6P7K2WPeY1dy
3,-1.067484,0.378228,1.735096,-0.896303,-0.665233,0,19,TUdiT-gEHdtkmr9JkhqvtNy5L_nutzsw2oH23kgO71NSsA
4,0.802854,-0.087691,-0.540774,0.072379,0.090969,0,145,qmcY1AJTdznK_aTgW2e1qwCY1uwbam9HU09Wsa7nI9Wt5sYQ
...,...,...,...,...,...,...,...,...
99995,-0.181216,0.817749,-0.076838,0.950896,0.118960,0,236,XU_3wTozp_g-sec3SdeBp7WOg61BU4Bsi9P636axectzPRY
99996,0.599617,0.275636,0.099153,0.543064,0.153125,0,245,oYgrmGD1cGqAN6My_talEYVTEp9YpZkjDsIZ3NMhjWqI9g
99997,-0.517921,0.477724,0.607669,-0.546329,-0.406754,0,432,q4uKpVa1RT8cihUV3Wa1t9Y7o-9CttHF3mOYxtdkksSUKQ
99998,0.642316,0.283726,0.252854,0.130415,-0.408584,0,55,Q_fw1SGj4BUHxM3gFnRWx7WcchwUTZbN4S-F7W0jwguxi-4


In [30]:
#plt.scatter(x_data['kill'][x_data['championid']==5] ,x_data['death'][x_data['championid']==5] ,s=1)

In [31]:
'''
from sklearn.cluster import KMeans
model = KMeans(n_clusters=10, init='k-means++', n_init=10, max_iter=300, tol=1e-4)
model.fit(x_data)
'''

"\nfrom sklearn.cluster import KMeans\nmodel = KMeans(n_clusters=10, init='k-means++', n_init=10, max_iter=300, tol=1e-4)\nmodel.fit(x_data)\n"

In [32]:
itemStr=['item' + str(num) for num in range(6)]

In [33]:
#item Task

total_item_dict = {}
total_item_number_dict = {}

for gameCnt in tqdm(range(gameNum)) : 
    for i in range(10) :
        summoner_itemDict = {}
        for it in itemStr :
            if summoner_itemDict.get(data[gameCnt]['participants'][i]['stats'][it])==None :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] = 1
            else :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] += 1
        
        for itemNum in summoner_itemDict.keys() :
            if summoner_itemDict[itemNum]>=2 :
                if itemNum!=0:
                    if total_item_dict.get(itemDict[itemNum]) == None :
                        total_item_dict[itemDict[itemNum]] =1
                        total_item_number_dict[itemDict[itemNum]] = {2:0,3:0,4:0,5:0,6:0}
                        total_item_number_dict[itemDict[itemNum]][summoner_itemDict[itemNum]]+=1

                    else : 
                        total_item_dict[itemDict[itemNum]] +=1
                        total_item_number_dict[itemDict[itemNum]][summoner_itemDict[itemNum]]+=1


total_item_dict


100%|██████████| 10000/10000 [00:00<00:00, 11086.53it/s]


{'Aegis of the Legion': 3,
 'Aether Wisp': 9,
 'Amplifying Tome': 141,
 'Ardent Censer': 1,
 'B. F. Sword': 14,
 "Bami's Cinder": 31,
 'Bilgewater Cutlass': 1,
 'Blasting Wand': 5,
 'Broken Stopwatch': 113,
 'Catalyst of Aeons': 3,
 "Caulfield's Warhammer": 2,
 'Chain Vest': 4,
 'Chalice of Harmony': 25,
 'Cloak of Agility': 28,
 'Cloth Armor': 252,
 'Crystalline Bracer': 18,
 'Cull': 16,
 'Dagger': 957,
 'Dark Seal': 65,
 "Dead Man's Plate": 1,
 "Death's Dance": 1,
 "Doran's Blade": 1024,
 "Doran's Ring": 343,
 "Doran's Shield": 6,
 'Duskblade of Draktharr': 2,
 'Elixir of Wrath': 1,
 "Executioner's Calling": 8,
 'Faerie Charm': 492,
 'Fiendish Codex': 4,
 'Forbidden Idol': 7,
 "Giant's Belt": 15,
 'Glacial Shroud': 6,
 'Guardian Angel': 5,
 'Hextech Revolver': 2,
 'Infinity Edge': 11,
 "Jaurim's Fist": 2,
 'Kindlegem': 27,
 'Kircheis Shard': 10,
 'Long Sword': 1218,
 'Lost Chapter': 2,
 "Mejai's Soulstealer": 4,
 'Morellonomicon': 1,
 'Needlessly Large Rod': 511,
 'Negatron Cloak': 5

In [34]:
overTwo = [2421,2424,2420]
          # 깨진 초시계 2개 , 초시계
overSix = [1006         , 1004      ,1027       ,  1028     , 1082]
            #원기회복구슬 ,요정의부적,사파이어 수정,루비 수정, 암흑의 인장
'''
for i in itemDict:
    if itemDict[i] == 'Rejuvenation Bead' :
        print(i)
'''     

"\nfor i in itemDict:\n    if itemDict[i] == 'Rejuvenation Bead' :\n        print(i)\n"

In [35]:
item_six_number = []
item_six_cnt = []
for key,value in sorted(total_item_dict.items() , key = lambda x : total_item_number_dict[x[0]][6] , reverse = True) :
    #print(key,value,total_item_number_dict[key][2],total_item_number_dict[key][3],total_item_number_dict[key][4],total_item_number_dict[key][5],total_item_number_dict[key][6])
    print(key,total_item_number_dict[key][6])


Rejuvenation Bead 214
Faerie Charm 82
Broken Stopwatch 46
Dark Seal 36
Sapphire Crystal 34
Stopwatch 19
Ruby Crystal 18
Doran's Ring 13
Kindlegem 13
Chalice of Harmony 12
Dagger 12
Cull 10
Cloth Armor 9
Bami's Cinder 9
Doran's Blade 8
Crystalline Bracer 7
Long Sword 7
Amplifying Tome 6
Kircheis Shard 6
Executioner's Calling 3
Doran's Shield 3
Glacial Shroud 3
Aether Wisp 3
Needlessly Large Rod 3
B. F. Sword 3
Null-Magic Mantle 2
Negatron Cloak 2
Vampiric Scepter 2
Catalyst of Aeons 2
Giant's Belt 1
Pickaxe 1
Spectre's Cowl 1
Forbidden Idol 1
Chain Vest 1
Mejai's Soulstealer 1
Elixir of Wrath 1
Quicksilver Sash 1
Cloak of Agility 1
Fiendish Codex 1
Serrated Dirk 1
Lost Chapter 0
Aegis of the Legion 0
Ardent Censer 0
Rabadon's Deathcap 0
Rapid Firecannon 0
Zeal 0
Infinity Edge 0
Titanic Hydra 0
Guardian Angel 0
Statikk Shiv 0
Youmuu's Ghostblade 0
Morellonomicon 0
Caulfield's Warhammer 0
Jaurim's Fist 0
Sterak's Gage 0
Zhonya's Hourglass 0
Void Staff 0
Dead Man's Plate 0
Umbral Glaive 0


In [36]:

pd.DataFrame()

""


In [37]:
# 각 아이템 구매한 유저들 번호 

troll_playerList = []
for gameCnt in tqdm(range(gameNum)) : 
    for i in range(10) :
        summoner_itemDict = {}
        for it in itemStr :
            if summoner_itemDict.get(data[gameCnt]['participants'][i]['stats'][it])==None :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] = 1
            else :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] += 1
        
        for itemNum in summoner_itemDict.keys() :
            if (itemNum in overTwo and summoner_itemDict[itemNum]>=2  ) or (itemNum in overSix and summoner_itemDict[itemNum]==6):
                troll_playerList.append([gameCnt,i])

troll_playerList

100%|██████████| 10000/10000 [00:01<00:00, 7886.45it/s]


[[6, 0],
 [13, 9],
 [22, 9],
 [30, 4],
 [35, 0],
 [37, 8],
 [43, 6],
 [56, 1],
 [62, 5],
 [82, 1],
 [91, 0],
 [129, 0],
 [153, 6],
 [157, 6],
 [161, 3],
 [162, 6],
 [168, 9],
 [198, 6],
 [222, 6],
 [222, 9],
 [233, 7],
 [237, 8],
 [262, 8],
 [298, 1],
 [312, 5],
 [314, 6],
 [327, 6],
 [327, 9],
 [335, 1],
 [335, 3],
 [345, 4],
 [346, 9],
 [357, 6],
 [370, 2],
 [379, 7],
 [382, 9],
 [409, 7],
 [414, 9],
 [442, 8],
 [454, 2],
 [507, 7],
 [508, 8],
 [509, 9],
 [513, 0],
 [534, 6],
 [536, 0],
 [548, 3],
 [582, 6],
 [622, 4],
 [624, 5],
 [631, 3],
 [633, 5],
 [666, 6],
 [667, 3],
 [687, 3],
 [697, 7],
 [719, 2],
 [719, 3],
 [724, 5],
 [730, 3],
 [751, 5],
 [785, 7],
 [819, 3],
 [828, 8],
 [829, 5],
 [848, 5],
 [855, 0],
 [878, 2],
 [881, 6],
 [881, 8],
 [929, 3],
 [964, 6],
 [973, 6],
 [982, 9],
 [1033, 2],
 [1073, 8],
 [1085, 0],
 [1089, 5],
 [1100, 8],
 [1116, 0],
 [1138, 5],
 [1138, 7],
 [1138, 9],
 [1145, 6],
 [1156, 1],
 [1168, 2],
 [1184, 7],
 [1205, 5],
 [1242, 4],
 [1258, 3],
 [1262

In [38]:
len(troll_playerList)

531

In [39]:
#dictionary에 troll_player들 넣어서 각 유저별 아이템 판매횟수 확인

troll_player_dict = {}

for game_num,player_cnt in troll_playerList :
    if troll_player_dict.get(data[game_num]['participantidentities'][player_cnt]['player']['summonerid']) == None :
        troll_player_dict[data[game_num]['participantidentities'][player_cnt]['player']['summonerid']] = 1
    else :
        troll_player_dict[data[game_num]['participantidentities'][player_cnt]['player']['summonerid']] += 1

troll_player_dict_sorted = sorted(troll_player_dict.items(),key = lambda x : x[1], reverse=True)

troll_player_dict_sorted[:20]

[('Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM', 8),
 ('2zz5blkjYNGp3ypIf2Rv7hlmgaFP0Lyfur8Ukzujua4zDQ', 4),
 ('3lZaLVQn3BXZIMrWRF2iWerxroM2l2A68XzMnZlKEF6p72k', 3),
 ('RSqzwOlH-hgS7kM1cUC9Uwhkt6VbZgcRT1Ei7FA-LBgdSg', 3),
 ('51CMgUaia2ZlTo8TkFv16pMN_TMXUF6ww8xDnJl60t8mJuo', 3),
 ('0M-qX85hegHj_x4YcMHTMH8qOFoD1LzCF5SMfIbCbJKWXtEk', 3),
 ('T6kOMf_mNkBLI9Xp9jyDOOCk3BNABZFVAPHbEf2ePIcvLI0', 3),
 ('5vCLowVZKtjWAIRwXBnSn_URa6IXuofykrPlA_RQsMOFmiE', 2),
 ('ymlzljXNA_w9OmCAlHg1Ns75s0phoHd1impHcs1nNEdjDN5y', 2),
 ('juNijhT5GlFctS2exDpU5IePH6cOECrAswzCvkbgcQfgwFw', 2),
 ('V4FkX5KuO7uY_OycSLuS2higoeNDlNQUrBFPHnmgZHaGsfg', 2),
 ('ZDGZuk8A8eEkqfoumh4Sgm4VXl-VixXbl7ArZm83S67cB4A', 2),
 ('XUzUPejBupr83KUVEcH5yh5zZHMuXyu58bwVHCTDzZFxI9Y', 2),
 ('mOsGKBOaxsQQx0EPiFJxQxHOu2f6I4iJgRp1dTUCfh6k4Ahg', 2),
 ('kW9Fg7SpHhVH2DTxf2alC8yx2Ks4sRipqJtaSIkj5CoVGN0', 2),
 ('EIZjUUsXJbQznvjnl0F9TtqlRNstoOzX6LJY1w9jVtHBq-E', 2),
 ('wWMFdZaEzk3xgpHZos326mUsrcbFfU_EbzxHVSI00uJxcw', 2),
 ('2qzOcd22gAk-zwBVPJcEX8JIAsvn

In [40]:
for i,item_num in enumerate(itemStr) : 
    x_data[item_num] = items[i]

In [51]:
type(df_shuffled_image_html)

str

In [ ]:
with open('df_shuffled_image_html.html', 'a') as df_shuffled_image_html:
   df_shuffled_image_html.write(html_text)

In [41]:
from IPython.core.display import HTML

most_troll_player_id = troll_player_dict_sorted[0][0]

df_shuffled_image = x_data[x_data['summoner_id'] == most_troll_player_id]

for item_num in tqdm(itemStr) :
    df_shuffled_image[item_num] = df_shuffled_image[item_num].astype(str)
    df_shuffled_image[item_num] = '<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/' + df_shuffled_image[item_num] + '.png\' weight=25 height=25>'

for item_num in tqdm(itemStr) :
    for i,item_img in enumerate(df_shuffled_image[item_num]) :
        if item_img == "<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/0.png\' weight=25 height=25>" :
            df_shuffled_image[item_num][df_shuffled_image.index[i]] = ''

df_shuffled_image_html=df_shuffled_image.to_html(escape=False)
#print(df_shuffled_image_html[:1600])
HTML(df_shuffled_image_html)




100%|██████████| 6/6 [00:00<00:00, 1200.14it/s]


,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
7515,2.173647,1.767286,-1.350843,2.678697,1.707086,0,82,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
8295,-0.955169,0.709283,-0.965989,0.282145,0.967431,0,350,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
34890,-1.101918,-1.588167,-0.964603,-2.027894,-3.399348,0,81,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
39785,0.589583,-1.515974,2.490203,-0.290494,-0.423415,0,235,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
40275,-1.392273,-1.683863,0.361523,-3.352880,-6.246304,0,51,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
43938,-0.231920,-0.731629,-0.074163,-1.208704,-1.738174,0,81,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
50828,-1.361806,-1.588167,-0.806331,-2.732676,-3.832546,0,81,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
52933,-1.164665,-1.001022,-0.632536,-1.621782,-1.183154,0,235,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
64090,-0.214889,0.430128,-1.451541,0.325548,0.549773,0,350,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
69787,-0.179052,1.058447,-1.732681,0.763133,0.056198,0,103,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,


In [42]:
print(x_data[x_data['summoner_id'] == most_troll_player_id]['kill'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['death'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['assist'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['gold'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['level'].mean())


-0.38384616056855597 -0.5126960483286783 -0.4143678177120142 -0.7184906737014559 -1.354245205461863


In [43]:
#troll_player_dict[-10:]
len(troll_player_dict)

487

In [44]:
pd.set_option('display.max_colwidth', 500)

In [45]:
# from IPython.core.display import HTML

# image_ex = pd.DataFrame([('http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/1001.png')])

# image_ex=image_ex.append(['http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/1004.png'] )

# image_ex[0] = '<img src = \'' + image_ex[0] + '\' weight=100 height=100>'

# image_html=image_ex.to_html(escape=False)

# HTML(image_html)

In [46]:
type(image_html)

NameError: name 'image_html' is not defined

In [44]:
for i in tqdm(range(len(troll_playerList))) :
    x_data['troll'][troll_playerList[i][0]*10 + troll_playerList[i][1]] = 1
x_data

100%|██████████| 2878/2878 [00:03<00:00, 768.91it/s]


,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
0,-0.354300,1.713000,-0.423267,-0.450153,-0.586478,0,38,PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA,1054,3003,2421,3020,3027,0
1,1.203082,0.250536,1.275325,0.467636,-0.207582,0,497,JRuCiFpuzcsnQ9Gadn5A_0bm66-PrFaA3WKP5juhgetrx3Y,2065,3109,3859,0,0,3117
2,0.913792,-0.223291,0.507604,0.442982,-0.516673,0,164,o5ZjlQkLgDZzDG_dkT0GVL7X34kp51abqkUN6P7K2WPeY1dy,1055,3105,3078,3158,1036,3074
3,-1.039395,0.316755,1.627199,-0.885815,-0.683338,0,19,TUdiT-gEHdtkmr9JkhqvtNy5L_nutzsw2oH23kgO71NSsA,1413,3748,2055,1028,3009,1029
4,0.826825,-0.105975,-0.524920,0.064029,0.080793,0,145,qmcY1AJTdznK_aTgW2e1qwCY1uwbam9HU09Wsa7nI9Wt5sYQ,1055,1052,3134,3508,3006,3115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,-0.647090,-1.675563,0.220612,-0.763886,-0.275209,0,76,MBeOxVqTseMAoDvr6pXpABqiWNgZPYj-xqTwsMTPQzpIzzs,1414,1052,3158,3100,1029,0
499996,-1.337254,-0.843469,0.322117,-1.838872,-1.022104,0,114,3ZK5oy4Sp1iXUCjGqZwVxxdipu2DDrHI14U9L0fXEEMC7P0,0,3074,0,0,0,0
499997,-0.366632,0.350473,1.505000,0.109395,1.359512,0,53,EqZgtCxYd5MqQiRypqDJCwGaGneuFF_5eQRo9wq3nJFW4Q,3857,3152,2031,3117,3024,0
499998,-0.137708,-0.207421,-0.328709,-0.779981,0.199627,0,238,a_dqCx7bUOtfGOjdF2n8o1XZ6FQNpa7PO6JPpzKWHSeBx4Y,3142,2055,3147,0,3047,0


In [45]:
troll_df = x_data[x_data['troll']==1]

troll_df

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
60,-0.474647,-0.902680,1.365034,-0.573393,-0.744094,1,22,zvgRBByyRKJi5Q1i49N2mcnNSr5IiWKxS5xakwE0_CZqKIs,2420,2420,2420,2420,2420,2420
139,-1.034070,-1.325805,1.988319,-1.753873,-0.627826,1,39,6Uz9KiY6ayRRCwhbA6Wc-prEFUfSSAhAmpevAGDprLm6sng,2421,2421,2421,2421,0,0
229,-1.539536,-1.518765,0.245974,-2.474451,-1.229963,1,2,p109-T2B_cR4sG_fzAdfyQlj_nWLL5wCCOnSJ00YJEzEP7oR,1006,1006,1006,1006,1006,1006
304,-0.787480,-0.177254,0.631486,-0.728422,-0.429515,1,60,VrPo9rsyWtihufnWTwXEF9mMAhzj2qbDvF0FPiZZ0WamQL8,1006,1006,1006,1006,1006,1006
350,-0.635912,-0.306484,0.777179,-0.158367,-0.204357,1,875,t8iVT_qOY-mxiRK72VdPxrF-eL8MnCRqc05gZDBYLf2ZzQ,1006,1006,1006,1006,1006,1006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499241,-0.373600,-1.111804,1.442895,-0.059352,0.009381,1,429,JQDcMZh5mXx0wlFY05Z1ygsBmACrouHksUAr9CmjSX5YI0o,1004,1004,1004,1004,1004,1004
499243,-1.291680,-0.942660,0.088953,-0.026466,0.563452,1,50,c60TXBRjMgFIoCVPAHZ6tR4_XXQgrhsmtGHEAgcfNjLEGeg,1006,1006,1006,1006,1006,1006
499519,-0.660758,-0.830439,1.528880,-1.300203,-1.293358,1,777,IsZ8l7AN26g-INH_KHk-7e29o7OLirU5dWFOYgXrPAOQcQ,1006,1006,1006,1006,1006,1006
499694,-0.975253,-1.454642,-0.738311,-1.240694,-1.339080,1,350,Xaze0GmS3K6xWsajaIUiI_G9YDpSWx9dtqTzuDMvVRwHqQ,1004,1004,1004,1004,1004,1004


In [46]:
print(troll_df['kill'].mean(),troll_df['death'].mean(),troll_df['assist'].mean(),troll_df['gold'].mean(),troll_df['level'].mean())

-0.6893137114760749 0.3289518119634393 -0.8736356613772907 -1.1722007629543592 -0.65827844976189


In [47]:
len(troll_playerList)

2878

In [48]:
#troll_df.drop('time',axis=1,inplace=True)
troll_df['troll'] = 1

In [53]:
sampling_df = x_data.sample(n=len(troll_playerList) * 9, random_state=1)
print(sampling_df['troll'].sum())
sampling_df.drop(sampling_df.loc[sampling_df['troll']==1].index, inplace=True)
print(sampling_df['troll'].sum())
sampling_df

146
0


,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
555024,-1.166051,-0.655031,-0.102720,-1.522117,-1.571323,0,235,_b4_Sgh07B91bBkyv9Hk02AEE0DaEBdgFT4S4bdRBZpXqg,1055,1037,3070,2422,0,0
236417,0.501077,-0.969794,0.855967,-0.036911,-0.101492,0,120,Nzf7P_hh-v0jeFsg50gbDG861e-g1jW475KoUINy9k9-jZw,1039,2031,3078,3047,3133,0
91618,-0.177311,0.328023,1.376530,-0.637070,-0.084618,0,80,8rKYMM1OAUaXBrmODj0mBS-d7gBelvrGb5_YoSvBVtEL1g,3179,3855,1036,2055,3047,1036
53904,-0.832790,-1.845416,-1.863930,-1.957696,-2.152697,0,80,s0Q2aiTMlaVDq0bA-fuBoKOmePLYCvJBTbDNCYWjnXqsdTc,3070,0,0,0,0,0
350187,-1.050806,-0.072569,-0.739909,-0.011964,-0.432174,0,12,f36ejsk8tTMrZQAGLAAtHyOU1QrVxeMEQ0lymn21CiGv5w,3860,3800,3108,3067,3117,3113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526203,0.322053,-0.930169,-1.728558,1.213042,1.970515,0,498,yKHKlYS2bCGuM9M_G1tzQTaZ3iSyMAw5CUmraeVX6Dfsycc,3086,3006,3508,3031,1042,0
149129,-0.127624,-0.073917,0.553909,-1.047823,0.081478,0,429,s3_YXJULpLVEcKyn7reh7QGa3pp-xitTwBlmqC2n5sje6-M,1055,3006,2055,3144,1043,1042
351030,3.291329,0.245089,-1.238109,2.377513,1.567857,0,235,HKsDfRQLoQwVGFW1Rsc8NG7fw6LCmuVHyIy71vkCGnMB2A,3094,3134,2421,3009,3071,3042
339659,-0.814732,0.549131,0.828246,-1.283795,-1.352466,0,98,axIr32wnDFPKwtFzg3GR9ySD-qfYC8G35CXglpDIrJy2zn4,0,3193,3859,1028,1029,3009


In [54]:
import tensorflow as tf
from tensorflow import keras

sampling_df2 = pd.concat([sampling_df,troll_df])

df_shuffled_total = sampling_df2.sample(frac=1,random_state=1).reset_index(drop=True)
df_shuffled = df_shuffled_total.drop(['championid','summoner_id'],axis=1)
df_shuffled = df_shuffled.drop(itemStr,axis=1)

y = df_shuffled['troll']
#print(df_shuffled['troll'].sum())
#df_shuffled
len(y)

34084

In [55]:
neg, pos = np.bincount(y)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 34084
    Positive: 3423 (10.04% of total)



In [56]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.

weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.56
Weight for class 1: 4.98


In [57]:
x = df_shuffled.drop(['troll'],axis=1)
train_data_len = len(x)*7//10
x_train = x[:train_data_len]
x_test = x[train_data_len:]
y_train = y[:train_data_len]
y_test = y[train_data_len:]

In [58]:
del data
del x_data

In [59]:
len(x_train)

23858

In [60]:
x_train 

,kill,assist,death,gold,level
0,-0.947593,0.851989,-1.267564,-0.580162,-0.640600
1,-0.497719,1.175502,0.294790,-0.454072,-0.923020
2,-0.222268,1.058626,0.378772,0.376246,0.204352
3,0.345960,-0.307407,-1.417761,0.230176,-0.066684
4,0.063659,0.673633,-0.051037,0.282353,-0.632253
...,...,...,...,...,...
23853,0.514691,1.327201,1.120122,-0.401208,-0.639210
23854,-1.095186,2.335536,-0.115074,0.215095,0.821914
23855,-1.410663,-1.669091,-1.677303,-2.208770,1.668378
23856,0.508850,1.347975,-0.112877,0.278641,-0.344424


In [61]:
model = keras.Sequential()
model.add(tf.keras.layers.Dense(5,input_shape=(5,),activation='relu'))
model.add(tf.keras.layers.Dense(5,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=['binary_accuracy',tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision')])


model.fit(x_train,y_train,epochs=20,batch_size=50,verbose=1,class_weight=class_weight)

Epoch 1/20
478/478 [==============================] - 1s 2ms/step - loss: 0.5405 - binary_accuracy: 0.6541 - Precision: 0.2038 - Recall: 0.8395
Epoch 2/20
478/478 [==============================] - 1s 2ms/step - loss: 0.5108 - binary_accuracy: 0.7253 - Precision: 0.2369 - Recall: 0.7795
Epoch 3/20
478/478 [==============================] - 1s 2ms/step - loss: 0.5072 - binary_accuracy: 0.7366 - Precision: 0.2452 - Recall: 0.7791
Epoch 4/20
478/478 [==============================] - 1s 2ms/step - loss: 0.5044 - binary_accuracy: 0.7387 - Precision: 0.2469 - Recall: 0.7795
Epoch 5/20
478/478 [==============================] - 1s 2ms/step - loss: 0.5024 - binary_accuracy: 0.7425 - Precision: 0.2486 - Recall: 0.7716
Epoch 6/20
478/478 [==============================] - 1s 2ms/step - loss: 0.5004 - binary_accuracy: 0.7468 - Precision: 0.2523 - Recall: 0.7732
Epoch 7/20
478/478 [==============================] - 1s 2ms/step - loss: 0.4995 - binary_accuracy: 0.7412 - Precision: 0.2493 - Recall:

In [62]:
model.evaluate(x_test,y_test,batch_size=50)

205/205 [==============================] - 0s 1ms/step - loss: 0.4887 - binary_accuracy: 0.7556 - Precision: 0.2572 - Recall: 0.7627


[0.48867714405059814, 0.755622923374176, 0.7626953125, 0.2571616768836975]

In [63]:
pred = model.predict(x_test)

In [64]:
pred[:20]

array([[0.0699679 ],
       [0.08977252],
       [0.8237741 ],
       [0.9377092 ],
       [0.37069494],
       [0.66409534],
       [0.44360366],
       [0.58676994],
       [0.56200635],
       [0.16511035],
       [0.156468  ],
       [0.35039592],
       [0.04809344],
       [0.81859934],
       [0.38566905],
       [0.32063365],
       [0.6128961 ],
       [0.14239848],
       [0.58029515],
       [0.27244014]], dtype=float32)

In [65]:
y_test[:20]

23858    0
23859    0
23860    0
23861    1
23862    0
23863    0
23864    0
23865    0
23866    1
23867    0
23868    0
23869    0
23870    0
23871    1
23872    0
23873    0
23874    0
23875    0
23876    0
23877    0
Name: troll, dtype: int64

In [258]:
from IPython.core.display import HTML

df_shuffled_image = df_shuffled_total[train_data_len:][(y_test==1) & (series_pred<0.5)][:30].drop(['summoner_id','troll_count'],axis=1)
df_shuffled_image

for item_num in tqdm(itemStr) :
    df_shuffled_image[item_num] = df_shuffled_image[item_num].astype(str)
    df_shuffled_image[item_num] = '<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/' + df_shuffled_image[item_num] + '.png\' weight=25 height=25>'

for item_num in tqdm(itemStr) :
    for i,item_img in enumerate(df_shuffled_image[item_num]) :
        if item_img == "<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/0.png\' weight=25 height=25>" :
            df_shuffled_image[item_num][df_shuffled_image.index[i]] = ''

df_shuffled_image_html=df_shuffled_image.to_html(escape=False)
#print(df_shuffled_image_html[:1600])
HTML(df_shuffled_image_html)


100%|██████████| 6/6 [00:00<00:00, 3002.72it/s]


,kill,assist,death,gold,level,troll,championid,item0,item1,item2,item3,item4,item5
23892,1.188684,3.013323,0.984122,0.225111,-1.050882,1,91,,,,,,
24053,1.207006,-1.357017,0.320586,0.563737,0.698133,1,80,,,,,,
24077,-0.323467,-0.405944,1.088179,-0.559149,-1.033992,1,517,,,,,,
24098,0.063402,-0.181606,-1.037052,-0.037530,-0.128689,1,80,,,,,,
24165,-0.559930,1.864688,0.200992,-0.409071,0.102344,1,11,,,,,,
24213,-0.488163,0.026460,0.229208,-0.439995,-0.538801,1,80,,,,,,
24231,-1.278346,-0.668653,-0.751727,-0.191620,0.693109,1,120,,,,,,
24255,0.252187,-0.596765,-0.414108,0.403689,-0.678331,1,122,,,,,,
24311,0.217005,1.992677,0.093604,-0.024714,-1.220291,1,103,,,,,,
24406,-1.112676,0.226922,1.053196,-0.245802,-0.635307,1,114,,,,,,


In [ ]:
df_shuffled_image = df_shuffled_total[train_data_len:][(y_test==0) & (series_pred>=0.5)][20:30].drop(['summoner_id','troll_count'],axis=1)
df_shuffled_image

for item_num in tqdm(itemStr) :
    df_shuffled_image[item_num] = df_shuffled_image[item_num].astype(str)
    df_shuffled_image[item_num] = '<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/' + df_shuffled_image[item_num] + '.png\' weight=25 height=25>'

for item_num in tqdm(itemStr) :
    for i,item_img in enumerate(df_shuffled_image[item_num]) :
        if item_img == "<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/0.png\' weight=25 height=25>" :
            df_shuffled_image[item_num][df_shuffled_image.index[i]] = ''

df_shuffled_image_html=df_shuffled_image.to_html(escape=False)
#print(df_shuffled_image_html[:1600])
HTML(df_shuffled_image_html)


In [66]:
df_shuffled_total[train_data_len:][pred>=0.5]

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
23860,-0.742625,-0.757071,-0.192173,-1.730272,0.115476,0,58,PPoTluxjv7nsMomGKCeQhs2itGd7rUyXeB1ASJ-Dqjb-Ky8,1054,3144,2031,1042,2055,0
23861,-1.456956,-1.025541,-1.114415,-2.818399,-3.994606,1,39,-SqIdIDjz_khyYEOMaoQLiJV9hIwQfV2MhFWbktWoJGq8rI,1004,1004,1004,1004,1004,1004
23863,-1.176764,-1.400523,0.787290,-0.931819,0.865851,0,84,zZAngBlzwqZ2sCfh0kTHl8hwWRgf01uld1jAeRQC1QH4BHE,3146,1052,1054,1001,0,0
23865,-0.315017,-1.443172,0.927456,-0.796068,1.450053,0,157,E7ad5qKBesJBePz_i648zxNKgwO7566W3vHpJIwqC7GMric,1055,2031,3086,1055,3006,0
23866,-0.385759,0.181926,2.306645,-1.216372,-1.221018,1,429,RU0pnr6luAZppK6zlamP-7byBiKiA4kEeWLg-cK5mQZyKXw,1004,1004,1004,1004,1004,1004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34067,-0.149350,-1.472972,1.101931,-1.745142,-0.521460,1,236,K_F-ikCrFEdg09hFNoKEpolg7L22iqS9Y-VcF43UTfjZ4g,1006,1006,1006,1006,1006,1006
34074,-1.062492,-0.838113,1.995214,-1.345389,0.107597,1,104,dLpRyFXB-SDk9steJawyS-Jf6hRv6cw3kxvmFIpO-XecsoM,1006,1006,1006,1006,1006,1006
34075,-0.462102,-0.722222,0.345234,-0.659270,0.309331,0,69,4Z9vKYWcO9yM5g8R8OrHcLRhiE7eEAnyFpfDR6tXupfryQ,3040,2003,3116,0,0,0
34077,-1.026829,-1.326531,1.460323,-1.784769,-0.550685,1,39,sVSawPUDveP1Lmf1FGIKv1bDg-Kb2MrMthTQcSxbF6Z057GG,2421,2421,2421,2421,0,0


In [67]:
[(y_test==0)][0][:10]

23858     True
23859     True
23860     True
23861    False
23862     True
23863     True
23864     True
23865     True
23866    False
23867     True
Name: troll, dtype: bool

In [68]:
list_pred = []
for p in pred :
    list_pred.append(p[0])

In [69]:
series_pred = pd.Series(list_pred)
y_test.index = [i for i in range(train_data_len,train_data_len+len(y_test))]
series_pred.index = [i for i in range(train_data_len,train_data_len+len(y_test))]

In [70]:
y_test

23858    0
23859    0
23860    0
23861    1
23862    0
        ..
34079    0
34080    1
34081    0
34082    0
34083    1
Name: troll, Length: 10226, dtype: int64

In [71]:
[(y_test==0) & (series_pred>=0.5)]

[23858    False
 23859    False
 23860     True
 23861    False
 23862    False
          ...  
 34079    False
 34080    False
 34081    False
 34082    False
 34083    False
 Length: 10226, dtype: bool]

In [84]:
troll_count = []
for summoner_id in df_shuffled_total['summoner_id'] :
    troll_count.append(  troll_player_dict.get(summoner_id) )
df_shuffled_total['troll_count'] = troll_count
df_shuffled_total.fillna(value=0,inplace=True)

In [87]:
df_shuffled_total[train_data_len:][(y_test==0) & (series_pred>=0.5)][20:30]

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5,troll_count
23958,-0.574912,0.155743,1.355545,-0.934616,-0.087305,0,142,iSmpHfRQHiWguEUm9lqiDHEkpn9dPkN8Enk6RnRX7tPnUG8,3157,2031,1056,1001,3285,0,0.0
23962,-0.995607,-1.687595,-1.392441,-2.026246,-4.956815,0,43,b03U7QMGPbYboVNAUJo3X8nflbSPDbuRNylF8IUvAA9Png,3850,0,0,0,0,0,0.0
23963,-1.400975,-1.439424,0.098945,-1.737542,1.199684,0,69,mwIlivMhixgAnZUPCKqgHGnxK-hwyx0mWpjYzjrrSD58Qw,2033,1082,3070,1056,3802,0,0.0
23965,0.459464,-1.785931,3.160320,-0.592500,0.312200,0,98,jmk_w2RvgyS4_Y3WZfT1O0lbFrUuoth0Vekm2LYwiaqj_i8,3748,2055,1054,1033,1001,0,0.0
23984,-0.943732,-0.391564,1.318360,-1.486425,0.386556,0,69,7bpktYtkj8r1Y4SufqxfHRWvMJWBosifIOehrHqaBh60rZA,1056,3070,2031,1082,0,3802,0.0
23985,-1.369390,-1.589085,-0.515048,-2.109652,-1.644368,0,81,QdgljfdoCWyPHYgE9WuoW16zutDbabk4Vm51dBvqqtuEzQ,2421,1001,1029,3057,3070,1055,0.0
23988,-0.450908,-0.490896,1.602401,-0.942134,-0.797476,0,17,gFiqZcCpBex3NNw3KRz7Uq6PJVmhVPMWAPGIGktoFtni1w,2031,1056,3136,3115,1001,0,0.0
23990,-0.938473,-0.515920,1.180931,-1.693229,-0.236060,0,84,W1S1niV2pG-MrBqWDKG2wzSFbkLff3qbbCDE5Pzp-5Hy5vM,3146,2055,3191,0,1054,0,0.0
23991,-1.324016,-1.587531,-1.791949,-2.344058,-2.970216,0,523,i3QD3-ItenHpw9q_gTeYPfyDLTGM3tbnrxjw45Zm83MV7Wc,0,0,0,0,0,0,0.0
23999,-1.565611,-1.723524,-0.936741,-2.806229,-2.711272,0,5,fX8MIbZE3_Mf0Y5i7bph5PGM2O9M_ZFeL1PGk6w2LGZHKqfB,1027,1004,1042,2055,1029,1006,0.0


In [86]:
df_shuffled_total[train_data_len:][(y_test==1) & (series_pred<0.5)][:20]

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5,troll_count
23892,1.188684,3.013323,0.984122,0.225111,-1.050882,1,91,t4Qf3CRjNM42zh2hGu-tQg8o95OdijorR1wewMFsKCctmxc,1004,1004,1004,1004,1004,1004,2.0
24053,1.207006,-1.357017,0.320586,0.563737,0.698133,1,80,qm_RNC9Afh4tZM3zIzFEvfNp1-IHnHP4mK5bkdQgvBmJyNF6,2420,2420,2420,2420,2420,2420,1.0
24077,-0.323467,-0.405944,1.088179,-0.559149,-1.033992,1,517,CBJIEViuGdgdtVkU0D1SmAcFg9VCLVukrBgustmMLyW7O6o,1027,1027,1027,1027,1027,1027,1.0
24098,0.063402,-0.181606,-1.037052,-0.037530,-0.128689,1,80,9ieAus2Owyea64jfRTKXj_iYQp2upssoHxHisfT9Cck3G_k,2420,2420,2420,2420,2420,2420,1.0
24165,-0.559930,1.864688,0.200992,-0.409071,0.102344,1,11,Slt0zsHvjMcAWbGc7hTjP8A3EN5UuiMHmapJlcd78PPVs68,1006,1006,1006,1006,1006,1006,2.0
24213,-0.488163,0.026460,0.229208,-0.439995,-0.538801,1,80,n2y571mwIoljDolgHuDpNc37K3lSp8Wl8rBv5AddLlNmrhs,2420,2420,2420,2420,2420,2420,1.0
24231,-1.278346,-0.668653,-0.751727,-0.191620,0.693109,1,120,7UsQqQ2zkhfLwjVpP-FBSbFGGL7XzhxMNNzUsxd4SBNj9BY,1004,1004,1004,1004,1004,1004,1.0
24255,0.252187,-0.596765,-0.414108,0.403689,-0.678331,1,122,ngLydBL8mqk-dcvJU6UdlJ_VpG7t21emOzXoEfaEG_S-CX4,2421,2421,2421,2421,2421,2421,1.0
24311,0.217005,1.992677,0.093604,-0.024714,-1.220291,1,103,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,1006,1006,1006,1006,1006,1006,29.0
24406,-1.112676,0.226922,1.053196,-0.245802,-0.635307,1,114,k_DKbZn4G04q2Nl01X_J1WicRq4uhLTuTHjojFVb1_RjazY,2420,2420,2420,2420,2420,2420,1.0
